In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import datetime
import numpy as np
from scipy.signal import argrelextrema
from scipy import signal

plt.rcParams['figure.figsize'] = (18, 8)

# Choose Time Interval

In [ ]:
start_date = datetime.date(2018,1,20)
end_date = datetime.date(2018,1,30)

# Load Water level data

In [ ]:
# load the water level data
path = "C:/Users/Marleen/Documents/thesis project/Data zaken/Data/Getij data/Waterhoogte Vlieland gemeten per 10 minuten.csv"
df_water_levels = pd.read_csv(path, delimiter=";")

# set date time to pandas timestamp 
df_water_levels['date_time'] = pd.to_datetime(df_water_levels['date_time'])

# drop location and obstype and tijd, datum
df_water_levels = df_water_levels[['date_time','waterheight']]

# select dates 
df_water_levels = df_water_levels.loc[(df_water_levels['date_time'].dt.date >= start_date) & (df_water_levels['date_time'].dt.date < end_date)]

# Load high/low tide time stamps

In [ ]:
# load the data
path = "C:/Users/Marleen/Documents/thesis project/Data zaken/Data/Getij data/Tijden Hoogwater en Laagwater Vlieland vanaf 2016.csv"
df_tide_times = pd.read_csv(path, delimiter=";")

# remove unnessary columns
df_tide_times = df_tide_times[['date_time', 'waterhoogte', 'extreem']]

# set date time to pandas timestamp 
df_tide_times['date_time'] = pd.to_datetime(df_tide_times['date_time'])

# select dates 
df_tide_times = df_tide_times.loc[(df_tide_times['date_time'].dt.date >= start_date) & (df_tide_times['date_time'].dt.date < end_date)]

# transform data from europe time to UTC
df_tide_times['date_time'] = df_tide_times['date_time'].dt.tz_localize('Europe/London').dt.tz_convert('UTC')

# round timestamps to intervals of 10 minutes
df_tide_times['date_time'] = df_tide_times['date_time'].dt.round('10min') 
df_tide_times.head()

# Plot some results

In [ ]:
plt.plot(df_water_levels['date_time'], df_water_levels['waterheight'])
plt.plot(df_tide_times[df_tide_times['extreem'] == 'HW'].date_time, df_tide_times[df_tide_times['extreem'] == 'HW'].waterhoogte, 'ro')

# Merge data sets and use waterheight from waterlevels data frame

The water levels from both data sets do not correspond perfectly, so choose the water levels in the water levels dataframe to work with.

In [ ]:
# convert to same date time format as water levels data
df_tide_times['date_time'] = df_tide_times['date_time'].values.astype('datetime64[ns]')

# perform left join on date time
df_water_levels_tides = pd.merge(df_water_levels, df_tide_times[['date_time','extreem']], how='left', on='date_time')

# make sure the data set starts and ends at high water
df_water_levels_tides = df_water_levels_tides.iloc[(df_water_levels_tides['extreem'] == 'HW').values.argmax():]
df_water_levels_tides = df_water_levels_tides.loc[:(df_water_levels_tides[df_water_levels_tides['extreem'] == 'HW']).last_valid_index()]

# create df with high/low water points 
df_high_water = df_water_levels_tides[df_water_levels_tides['extreem'] == 'HW']
df_low_water = df_water_levels_tides[df_water_levels_tides['extreem'] == 'LW']

# create a plot with water levels and high/low tide points
plt.plot(df_water_levels['date_time'], df_water_levels['waterheight'])
plt.plot(df_high_water.date_time, df_high_water.waterheight, 'ro')
plt.plot(df_low_water.date_time, df_low_water.waterheight, 'go')

# Calculate the length of the tidal cycle and the length until low tide

For every tidal cycle we want to know how long the cycle lasts (from high tide to the next high tide) and also the time it takes before low tide is reached (from high tide to low tide).

The time between tidal cycles is calculated as the number of steps per tidal cycle.

(!) Note that if the number of minutes within a time step is changed, this part has to be adapted as well! 

In [ ]:
# create extra column in df
df_water_levels_tides['time_steps_in_cycle'] = np.nan
df_water_levels_tides['time_steps_to_low_tide'] = np.nan

# get all indices with high water (easy now, see above code block)
high_water_indices = (df_water_levels_tides[df_water_levels_tides['extreem'] == 'HW']).index
low_water_indices =  (df_water_levels_tides[df_water_levels_tides['extreem'] == 'LW']).index

# number of time steps between high tides
num_time_steps_high_tides = np.diff(high_water_indices)

# calculate time steps till low tide
time_steps_to_low_tide = low_water_indices - high_water_indices[:-1]  

# couple number of time steps to high water time points (exclude the last one, since this point is the end of the simulation)
df_water_levels_tides.time_steps_in_cycle.loc[high_water_indices[:-1]] = num_time_steps_high_tides

# also add number of time steps to low tide for every high tide
df_water_levels_tides.time_steps_to_low_tide.loc[high_water_indices[:-1]] = time_steps_to_low_tide

In [ ]:
time_steps_low_tide

In [ ]:
df_water_levels_tides[70:120]

In [ ]:
102 - 64

In [ ]:
df['z'] = 0
# df.y.loc[df.x == 2]
df.z.iloc[3] = 1
df

In [ ]:
df.iloc[((df['x'] == xEnd) & (df['y'] == yEnd)).values.argmax():]

In [ ]:
((df['x'] == xEnd) & (df['y'] == yEnd)).last_valid_index()